# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# Import libraries
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, \
                                  avg, from_unixtime, split, min, max, round, lit, mean
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

#from pyspark.sql.types import IntegerType, TimestampType
import datetime
from pyspark.sql.functions import to_date, year, month, dayofmonth, dayofweek, hour, date_format, substring

import numpy as np
import pandas as pd
import time
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
# Create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

In [3]:
# Set time parser policy
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [5]:
# Load dataset
path = "data/mini_sparkify_event_data.json"
data = spark.read.json(path)
original_count = data.count()

### Common Functions

In [7]:
# Set plot's figure size
def set_plot_size(width, height):
    return plt.figure(figsize = [width, height]);

def get_user_logs(userId, sessionId=None):
    if sessionId == None:
        return data.where(data.userId == userId) \
            .select('tsDate', 'userId', 'sessionId', 'itemInSession', 'level', 'page') \
            .sort('tsDate', 'itemInSession')
    else:
        return data.where((data.userId == userId) & (data.sessionId == sessionId)) \
            .select('tsDate', 'userId', 'sessionId', 'itemInSession', 'level', 'page') \
            .sort('tsDate', 'itemInSession')
    
def get_users(churn):
    return data.where(data.churn == churn).select('userId').dropDuplicates()

# Load and prepare Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [8]:
# Remove rows with missing users
data = data.where(~((col('userId').isNull()) | (col('userId') == '')))

# Exclude non-relevant columns
data = data.drop('firstName')
data = data.drop('lastName')

data = data.withColumn('regDate', (col('registration') / 1000.0).cast(TimestampType()))
data = data.withColumn('tsDate', (col('ts') / 1000.0).cast(TimestampType()))
data.take(1)

data = data.withColumn('city', split(data['location'], ',')[0])
data = data.withColumn('state', split(data['location'], ',')[1])
data = data.drop('location')

# Define churned users using Cancellation Confirmation event (canceled)
query_churn_by_cc = data.where(data.page == 'Cancellation Confirmation')
print(f'Churned users who cancelled subscription: {query_churn_by_cc.count()}')

# Label churned (canceled) users
canceled = query_churn_by_cc.select('userId').dropDuplicates().select('userId')
canceled_uids = [row.userId for row in canceled.collect()];
set_churn = udf(lambda x: 1 if x in canceled_uids else 0, IntegerType())
data = data.withColumn('churn', set_churn('userId'))

# Add [userRowId] column that assigns a 1-based index to every user's log ordered by [ts]
w =  Window.partitionBy(data.userId).orderBy('ts', 'itemInSession')
data = data.withColumn('userRowId', row_number().over(w))

# Add [userRowDescId] column that assigns a 1-based index to every user's log ordered by [ts] descending.
w =  Window.partitionBy(data.userId).orderBy(col('ts').desc(), col('itemInSession').desc())
data = data.withColumn('userRowDescId', row_number().over(w))

# Add last level column
last_levels = dict()
for row in data.where(data.userRowDescId == 1).select('userId', 'level').collect():
    last_levels[row.userId] = row.level
get_level = udf(lambda userId: last_levels[userId])
data = data.withColumn('lastLevel', get_level('userId'))

Churned users who cancelled subscription: 52


In [9]:
users = data.select('churn', 'userId').dropDuplicates()

In [11]:
get_users(1).count()

52

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [12]:
labels = data.select(col('churn').alias('label'), 'userId').dropDuplicates()

### User Attribute Features

In [99]:
f_Gender = data \
    .select('userId', 'gender') \
    .dropDuplicates() \
    .replace(['M', 'F'], ['0', '1'], 'gender') \
    .select('userId', col('gender').cast('int').alias('Gender'))

In [100]:
f_LastLevel = data \
    .select('userId', 'lastLevel') \
    .dropDuplicates() \
    .replace(['free', 'paid'], ['0', '1'], 'lastLevel') \
    .select('userId', col('lastLevel').cast('int').alias('LastLevel'))

### Per Log Features

In [101]:
def page_count(page):
    return data \
        .where(data.page == page) \
        .groupby('userId') \
        .agg(count('userId').alias('count')) \
        .select('userId', col('count').alias(page.replace(' ', '') + 'Count'))

In [102]:
f_LogCount = data \
    .groupby('userId') \
    .agg(count('userId').alias('LogCount'))

In [103]:
f_SongCount = data \
    .where(data.page == 'NextSong') \
    .groupby('userId') \
    .agg(count('userId').alias('SongCount'))

In [104]:
f_NonSongCount = data \
    .where(data.page != 'NextSong') \
    .groupby('userId') \
    .agg(count('userId').alias('NonSongCount'))

In [105]:
f_AboutCount = page_count('About')

In [106]:
f_ThumbsUpCount = page_count('Thumbs Up')

In [107]:
f_RollAdvertCount = page_count('Roll Advert')

### Per Session Features

In [108]:
page_data = data.where(~data.page.isin(['Cancel', 'Cancellation Confirmation'])) \
    .select('page', 'userId', 'sessionId', 'ts')

In [109]:
# Average page count per session hour
session_hours = page_data \
    .groupby('userId', 'sessionId') \
    .agg(((max('ts') - min('ts'))/1000/3600).alias('sessionHours'))

def page_session_hour(page):
    return page_data \
        .where(data.page == page) \
        .join(session_hours, ['userId', 'sessionId'], 'inner') \
        .groupby( 'userId', 'sessionId', 'sessionHours') \
        .agg((count('userId')/col('sessionHours')).alias('avgPerSession')) \
        .groupby('userId') \
        .agg(avg('avgPerSession').alias('avg')) \
        .select('userId', col('avg').alias(page.replace(' ', '') + 'PerSessionHour'))

In [110]:
# Average page count per hour
user_hours = page_data \
    .groupby('userId', 'sessionId') \
    .agg(((max('ts') - min('ts'))/1000/3600).alias('sessionHours')) \
    .groupby('userId') \
    .agg(Fsum('sessionHours').alias('hours'))

def page_hour(page):
    return page_data \
        .where(data.page == page) \
        .join(user_hours, 'userId', 'inner') \
        .groupby('userId', 'hours') \
        .agg((count('userId')/col('hours')).alias('avg')) \
        .select('userId', col('avg').alias(page.replace(' ', '') + 'PerHour'))

In [111]:
f_SessionCount = data \
    .select('userId', 'sessionId') \
    .dropDuplicates() \
    .groupby('userId') \
    .agg(count('userId').alias('SessionCount'))

In [112]:
f_AvgSessionLength = data \
    .groupby('userId', 'sessionId') \
    .agg(((max('ts') - min('ts'))/1000).alias('sessionLength')) \
    .groupby('userId') \
    .agg(avg('sessionLength').alias('AvgSessionLength')) \

In [113]:
# Session gap
users = data.select('userId').dropDuplicates()

f_AvgSessionGap = data \
    .groupby('userId', 'sessionId') \
    .agg(min('ts').alias('startTime'), max('ts').alias('endTime')) \
    .groupby('userId') \
    .agg(count('userId').alias('sessionCount'), \
        ((max('endTime') - min('startTime'))/1000).alias('observationPeriodTime'), \
        (Fsum(col('endTime') - col('startTime'))/1000).alias('totalSessionTime')) \
    .where(col('sessionCount') > 1) \
    .join(users, 'userId', 'outer') \
    .fillna(0) \
    .select('userId', \
            (col('observationPeriodTime') - col('totalSessionTime')/(col('sessionCount') - 1)).alias('AvgSessionGap'))

In [114]:
f_DowngradePerSessionHour = page_session_hour('Downgrade')

In [115]:
f_ErrorPerSessionHour = page_session_hour('Error')

In [116]:
f_SettingsPerSessionHour = page_session_hour('Settings')

In [117]:
f_SaveSettingsPerSessionHour = page_session_hour('Save Settings')

In [118]:
f_LogoutPerSessionHour = page_session_hour('Logout')

In [119]:
f_SubmitDowngradePerSessionHour = page_session_hour('Submit Downgrade')

### Per Hour Features

In [120]:
f_RollAdvertPerHour = page_hour('Roll Advert')

In [121]:
f_ThumbsDownPerHour = page_hour('Thumbs Down')

In [122]:
f_UpgradePerHour = page_hour('Upgrade')

In [123]:
f_SubmitUpgradePerHour = page_hour('Submit Upgrade')

### Per Day Features

In [124]:
# Average page count per day
page_data = data \
    .where(~data.page.isin(['Cancel', 'Cancellation Confirmation'])) \
    .select('userId', 'page', 'date')

def page_day(page):
    return page_data \
        .where(data.page == page) \
        .groupby('userId', 'date') \
        .count() \
        .groupby('userId') \
        .agg(avg('count').alias(page.replace(' ', '') + 'PerDay'))

In [125]:
# Average sessions per day
f_SessionsPerDay = data \
    .select('userId', 'date', 'sessionId') \
    .dropDuplicates() \
    .groupby('userId', 'date') \
    .count() \
    .groupby('userId') \
    .agg(avg('count').alias('SessionsPerDay'))

In [126]:
f_AddFriendPerDay = page_day('Add Friend')

In [127]:
f_RollAdvertPerDay = page_day('Roll Advert')

In [128]:
f_ThumbsDownPerDay = page_day('Thumbs Down')

In [129]:
f_ThumbsUpPerDay = page_day('Thumbs Up')

### Per Song Features

In [130]:
# Total song length 
f_TotalSongLength = data \
    .where(data.page == 'NextSong') \
    .select('userId', 'length') \
    .groupby('userId') \
    .agg(Fsum('length').alias('TotalSongLength'))

In [131]:
# Unique song count
f_UniqueSongCount = data \
    .where(data.page == 'NextSong') \
    .select('userId', 'song') \
    .dropDuplicates() \
    .groupby('userId') \
    .agg(count('userId').alias('UniqueSongCount'))

In [132]:
# Share of unique songs among all user's songs
totals = data \
    .where(data.page == 'NextSong') \
    .select('userId') \
    .groupby('userId') \
    .agg(count('userId').alias('total'))

f_UniqueSongShare = data \
    .where(data.page == 'NextSong') \
    .select('userId', 'song') \
    .dropDuplicates() \
    .groupby('userId') \
    .count() \
    .join(totals, on = ['userId'], how = 'inner') \
    .select('userId', (col('count')/col('total')).alias('UniqueSongShare')) 

### model_data

In [148]:
# model 7b (churn #1, seed 0)
model_data  = labels.join(f_LogCount, 'userId', 'outer') \
    .join(f_SongCount, 'userId', 'outer') \
    .join(f_NonSongCount, 'userId', 'outer') \
    .join(f_AboutCount, 'userId', 'outer') \
    .join(f_ThumbsUpCount, 'userId', 'outer') \
    .join(f_RollAdvertCount, 'userId', 'outer') \
    .join(f_SessionCount, 'userId', 'outer') \
    .join(f_AvgSessionLength, 'userId', 'outer') \
    .join(f_AvgSessionGap, 'userId', 'outer') \
    .join(f_DowngradePerSessionHour, 'userId', 'outer') \
    .join(f_SettingsPerSessionHour, 'userId', 'outer') \
    .join(f_LogoutPerSessionHour, 'userId', 'outer') \
    .join(f_RollAdvertPerHour, 'userId', 'outer') \
    .join(f_ThumbsDownPerHour, 'userId', 'outer') \
    .join(f_SubmitUpgradePerHour, 'userId', 'outer') \
    .join(f_SessionsPerDay, 'userId', 'outer') \
    .join(f_AddFriendPerDay, 'userId', 'outer') \
    .join(f_RollAdvertPerDay, 'userId', 'outer') \
    .join(f_ThumbsDownPerDay, 'userId', 'outer') \
    .join(f_ThumbsUpPerDay, 'userId', 'outer') \
    .join(f_TotalSongLength, 'userId', 'outer') \
    .join(f_UniqueSongCount, 'userId', 'outer') \
    .join(f_UniqueSongShare, 'userId', 'outer') \
    .drop('userId') \
    .fillna(0)

In [161]:
# model 7 (churn #1, seed 0)
model_data_rf  = labels.join(f_Gender, 'userId', 'outer') \
    .join(f_LastLevel, 'userId', 'outer') \
    .join(f_LogCount, 'userId', 'outer') \
    .join(f_SongCount, 'userId', 'outer') \
    .join(f_NonSongCount, 'userId', 'outer') \
    .join(f_AboutCount, 'userId', 'outer') \
    .join(f_ThumbsUpCount, 'userId', 'outer') \
    .join(f_RollAdvertCount, 'userId', 'outer') \
    .join(f_SessionCount, 'userId', 'outer') \
    .join(f_AvgSessionLength, 'userId', 'outer') \
    .join(f_AvgSessionGap, 'userId', 'outer') \
    .join(f_DowngradePerSessionHour, 'userId', 'outer') \
    .join(f_ErrorPerSessionHour, 'userId', 'outer') \
    .join(f_SettingsPerSessionHour, 'userId', 'outer') \
    .join(f_SaveSettingsPerSessionHour, 'userId', 'outer') \
    .join(f_LogoutPerSessionHour, 'userId', 'outer') \
    .join(f_SubmitDowngradePerSessionHour, 'userId', 'outer') \
    .join(f_RollAdvertPerHour, 'userId', 'outer') \
    .join(f_ThumbsDownPerHour, 'userId', 'outer') \
    .join(f_UpgradePerHour, 'userId', 'outer') \
    .join(f_SubmitUpgradePerHour, 'userId', 'outer') \
    .join(f_SessionsPerDay, 'userId', 'outer') \
    .join(f_AddFriendPerDay, 'userId', 'outer') \
    .join(f_RollAdvertPerDay, 'userId', 'outer') \
    .join(f_ThumbsDownPerDay, 'userId', 'outer') \
    .join(f_ThumbsUpPerDay, 'userId', 'outer') \
    .join(f_TotalSongLength, 'userId', 'outer') \
    .join(f_UniqueSongCount, 'userId', 'outer') \
    .join(f_UniqueSongShare, 'userId', 'outer') \
    .drop('userId') \
    .fillna(0)

In [177]:
# model 6
model_data  = labels.join(f_EventCount, 'userId', 'outer') \
    .join(f_AvgSessionCount, 'userId', 'outer') \
    .join(f_RollAdvertCount, 'userId', 'outer') \
    .join(f_ThumbsUpCount, 'userId', 'outer') \
    .join(f_RollAdvertCountPerHour, 'userId', 'outer') \
    .join(f_ThumbsDownCountPerHour, 'userId', 'outer') \
    .join(f_SettingsCountPerHour, 'userId', 'outer') \
    .join(f_LogoutCountPerHour, 'userId', 'outer') \
    .join(f_SaveSettingsCountPerHour, 'userId', 'outer') \
    .join(f_TotalSongLength, 'userId', 'outer') \
    .join(f_AvgNonSongEventCount, 'userId', 'outer') \
    .join(f_UniqueSongCount, 'userId', 'outer') \
    .join(f_UniqueSongShare, 'userId', 'outer') \
    .join(f_AvgGapTime, 'userId', 'outer') \
    .join(f_ThumbsDownCountPerDay, 'userId', 'outer') \
    .join(f_ThumbsUpCountPerDay, 'userId', 'outer') \
    .join(f_AvgSessionCountPerDay, 'userId', 'outer') \
    .drop('userId') \
    .fillna(0)

In [148]:
# model 5
model_data  = labels.join(f_EventCount, 'userId', 'outer') \
    .join(f_AvgSessionCount, 'userId', 'outer') \
    .join(f_RollAdvertCount, 'userId', 'outer') \
    .join(f_SubmitUpgradeCount, 'userId', 'outer') \
    .join(f_ThumbsUpCount, 'userId', 'outer') \
    .join(f_RollAdvertCountPerHour, 'userId', 'outer') \
    .join(f_SubmitUpgradeCountPerHour, 'userId', 'outer') \
    .join(f_ThumbsDownCountPerHour, 'userId', 'outer') \
    .join(f_UpgradeCountPerHour, 'userId', 'outer') \
    .join(f_SettingsCountPerHour, 'userId', 'outer') \
    .join(f_LogoutCountPerHour, 'userId', 'outer') \
    .join(f_SaveSettingsCountPerHour, 'userId', 'outer') \
    .join(f_TotalSongLength, 'userId', 'outer') \
    .join(f_AvgNonSongEventCount, 'userId', 'outer') \
    .join(f_UniqueSongCount, 'userId', 'outer') \
    .join(f_UniqueSongShare, 'userId', 'outer') \
    .join(f_AvgGapTime, 'userId', 'outer') \
    .join(f_ThumbsDownCountPerDay, 'userId', 'outer') \
    .join(f_ThumbsUpCountPerDay, 'userId', 'outer') \
    .join(f_UpgradeCountPerDay, 'userId', 'outer') \
    .drop('userId') \
    .fillna(0)

In [ ]:
# model 4
model_data  = labels.join(f_EventCount, 'userId', 'outer') \
    .join(f_AvgSessionCount, 'userId', 'outer') \
    .join(f_RollAdvertCount, 'userId', 'outer') \
    .join(f_SubmitUpgradeCount, 'userId', 'outer') \
    .join(f_ThumbsUpCount, 'userId', 'outer') \
    .join(f_RollAdvertCountPerHour, 'userId', 'outer') \
    .join(f_SubmitUpgradeCountPerHour, 'userId', 'outer') \
    .join(f_ThumbsDownCountPerHour, 'userId', 'outer') \
    .join(f_UpgradeCountPerHour, 'userId', 'outer') \
    .join(f_SettingsCountPerHour, 'userId', 'outer') \
    .join(f_LogoutCountPerHour, 'userId', 'outer') \
    .join(f_SaveSettingsCountPerHour, 'userId', 'outer') \
    .join(f_TotalSongLength, 'userId', 'outer') \
    .join(f_AvgNonSongEventCount, 'userId', 'outer') \
    .join(f_UniqueSongCount, 'userId', 'outer') \
    .join(f_UniqueSongShare, 'userId', 'outer') \
    .join(f_AvgGapTime, 'userId', 'outer') \
    .drop('userId') \
    .fillna(0)

In [130]:
# model 3
model_data  = labels.join(f_EventCount, 'userId', 'outer') \
    .join(f_AvgSessionCount, 'userId', 'outer') \
    .join(f_RollAdvertCount, 'userId', 'outer') \
    .join(f_SubmitUpgradeCount, 'userId', 'outer') \
    .join(f_ThumbsUpCount, 'userId', 'outer') \
    .join(f_RollAdvertCountPerHour, 'userId', 'outer') \
    .join(f_SubmitUpgradeCountPerHour, 'userId', 'outer') \
    .join(f_ThumbsDownCountPerHour, 'userId', 'outer') \
    .join(f_UpgradeCountPerHour, 'userId', 'outer') \
    .join(f_SettingsCountPerHour, 'userId', 'outer') \
    .join(f_LogoutCountPerHour, 'userId', 'outer') \
    .join(f_SaveSettingsCountPerHour, 'userId', 'outer') \
    .join(f_TotalSongLength, 'userId', 'outer') \
    .join(f_AvgNonSongEventCount, 'userId', 'outer') \
    .join(f_UniqueSongCount, 'userId', 'outer') \
    .join(f_UniqueSongShare, 'userId', 'outer') \
    .drop('userId') \
    .fillna(0)

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [279]:
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler, MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier, \
    NaiveBayes, DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [301]:
def split_data(data):
    train, test = data.randomSplit([0.8, 0.2], seed = 0);
    return train, test


def create_pipeline(data, classifier, scaler):
    features = data.drop('label').columns
    assembler = VectorAssembler(inputCols=features, outputCol='NumFeatures')
    pipeline = Pipeline(stages=[assembler, scaler, classifier]);

    return pipeline


def fit_model(name, data, classifier, scaler):
    
    start_time = time.time()
    
    print('Model fitting started. Please wait...')

    # Split data into train and test subset
    train, test = split_data(data)
    print('Train|Test data split done.')

    # Create pipeline
    pipeline = create_pipeline(data, classifier, scaler)
    print('Create pipeline done.')

    # train model
    model = pipeline.fit(train)
    print('Training done.')

    # metrics
    report(name, model, test)

    fitting_duration = time.time() - start_time
    
    print(f'Model fitting done ({int(fitting_duration)} s).')
    
    return model

In [271]:
# source: https://stackoverflow.com/questions/60772315/how-to-evaluate-a-classifier-with-apache-spark-2-4-5-and-pyspark-python
def report(name, model, test):

    # Create both evaluators
    evaluatorMulti = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction')
    evaluator = BinaryClassificationEvaluator(labelCol='label', rawPredictionCol='prediction', metricName='areaUnderROC')

    # Make predicitons
    prediction = model.transform(test).select('label', 'prediction')

    # Get metrics
    acc = evaluatorMulti.evaluate(prediction, {evaluatorMulti.metricName: 'accuracy'})
    f1 = evaluatorMulti.evaluate(prediction, {evaluatorMulti.metricName: 'f1'})
    weightedPrecision = evaluatorMulti.evaluate(prediction, {evaluatorMulti.metricName: 'weightedPrecision'})
    weightedRecall = evaluatorMulti.evaluate(prediction, {evaluatorMulti.metricName: 'weightedRecall'})
    auc = evaluator.evaluate(prediction)
    
    # show
    print(f'Metrics - {name}')
    print('-------------------------------------------------------')
    print(f' - accuracy: {acc}')
    print(f' - F1: {f1}')
    print(f' - weighted precision: {weightedPrecision}')
    print(f' - weighted recall: {weightedRecall}')
    print(f' - AUC: {auc}')
    print('-------------------------------------------------------')
    

### Logistic Regression

In [156]:
# Fit model
start_lr = time.time()
model_lr_fitted = pipeline_lr.fit(train);
end_lr = time.time()
lr_time = end_lr - start_lr

print('Total training time for logistic regression: {} seconds'.format(lr_time))

Total training time for logistic regression: 501.87249088287354 seconds


In [157]:
# model 7b
lr_f1 = eval_model(model_lr_fitted, test,"f1")
lr_acc = eval_model(model_lr_fitted, test,"accuracy")
print(f'Accuracy: {lr_acc}\nF1 score: {lr_f1}'

Logistic Regression - Accuracy: 0.86 Logistic Regression - F1 score: 0.8645800176834659


In [144]:
# model 7
lr_f1 = eval_model(model_lr_fitted, test,"f1")
lr_acc = eval_model(model_lr_fitted, test,"accuracy")
print('Logistic Regression - Accuracy: {}'.format(lr_acc),'Logistic Regression - F1 score: {}'.format(lr_f1))

Logistic Regression - Accuracy: 0.82 Logistic Regression - F1 score: 0.8348235294117647


In [56]:
# Small model
lr_f1 = eval_model(model_lr_fitted, test,"f1")
lr_acc = eval_model(model_lr_fitted, test,"accuracy")
print('Logistic Regression - Accuracy: {}'.format(lr_acc),'Logistic Regression - F1 score: {}'.format(lr_f1))

Logistic Regression - Accuracy: 0.7166666666666667 Logistic Regression - F1 score: 0.6682596575069694


In [73]:
# Extended model
lr_f1 = eval_model(model_lr_fitted, test,"f1")
lr_acc = eval_model(model_lr_fitted, test,"accuracy")
print('Logistic Regression - Accuracy: {}'.format(lr_acc),'Logistic Regression - F1 score: {}'.format(lr_f1))

Logistic Regression - Accuracy: 0.7666666666666667 Logistic Regression - F1 score: 0.7542179662562699


In [107]:
# model 2
lr_f1 = eval_model(model_lr_fitted, test,"f1")
lr_acc = eval_model(model_lr_fitted, test,"accuracy")
print('Logistic Regression - Accuracy: {}'.format(lr_acc),'Logistic Regression - F1 score: {}'.format(lr_f1))

Logistic Regression - Accuracy: 0.6935483870967742 Logistic Regression - F1 score: 0.6575340563509366


In [119]:
# model 3
lr_f1 = eval_model(model_lr_fitted, test,"f1")
lr_acc = eval_model(model_lr_fitted, test,"accuracy")
print('Logistic Regression - Accuracy: {}'.format(lr_acc),'Logistic Regression - F1 score: {}'.format(lr_f1))

Logistic Regression - Accuracy: 0.6935483870967742 Logistic Regression - F1 score: 0.6412688470314528


In [137]:
# model 4 (with session gap)
lr_f1 = eval_model(model_lr_fitted, test,"f1")
lr_acc = eval_model(model_lr_fitted, test,"accuracy")
print('Logistic Regression - Accuracy: {}'.format(lr_acc),'Logistic Regression - F1 score: {}'.format(lr_f1))

Logistic Regression - Accuracy: 0.8709677419354839 Logistic Regression - F1 score: 0.8525674199298487


In [155]:
# model 5
lr_f1 = eval_model(model_lr_fitted, test,"f1")

lr_acc = eval_model(model_lr_fitted, test,"accuracy")
print('Logistic Regression - Accuracy: {}'.format(lr_acc),'Logistic Regression - F1 score: {}'.format(lr_f1))

Logistic Regression - Accuracy: 0.8387096774193549 Logistic Regression - F1 score: 0.8295943687648757


In [168]:
# model 6
lr_f1 = eval_model(model_lr_fitted, test,"f1")
lr_acc = eval_model(model_lr_fitted, test,"accuracy")
print('Logistic Regression - Accuracy: {}'.format(lr_acc),'Logistic Regression - F1 score: {}'.format(lr_f1))

Logistic Regression - Accuracy: 0.8548387096774194 Logistic Regression - F1 score: 0.8300747170148985


In [183]:
# model 6 - churn #1
lr_f1 = eval_model(model_lr_fitted, test,"f1")
lr_acc = eval_model(model_lr_fitted, test,"accuracy")
print('Logistic Regression - Accuracy: {}'.format(lr_acc),'Logistic Regression - F1 score: {}'.format(lr_f1))

Logistic Regression - Accuracy: 0.8548387096774194 Logistic Regression - F1 score: 0.8300747170148985


In [190]:
# model 6 - churn #1 (seed=0)
lr_f1 = eval_model(model_lr_fitted, test,"f1")
lr_acc = eval_model(model_lr_fitted, test,"accuracy")
print('Logistic Regression - Accuracy: {}'.format(lr_acc),'Logistic Regression - F1 score: {}'.format(lr_f1))

Logistic Regression - Accuracy: 0.86 Logistic Regression - F1 score: 0.8645800176834659


## Random Forest

In [292]:
# best rf features (churn #1, seed 0)
data_rfc = labels.join(f_Gender, 'userId', 'outer') \
    .join(f_LastLevel, 'userId', 'outer') \
    .join(f_LogCount, 'userId', 'outer') \
    .join(f_SongCount, 'userId', 'outer') \
    .join(f_NonSongCount, 'userId', 'outer') \
    .join(f_AboutCount, 'userId', 'outer') \
    .join(f_ThumbsUpCount, 'userId', 'outer') \
    .join(f_RollAdvertCount, 'userId', 'outer') \
    .join(f_SessionCount, 'userId', 'outer') \
    .join(f_AvgSessionLength, 'userId', 'outer') \
    .join(f_AvgSessionGap, 'userId', 'outer') \
    .join(f_DowngradePerSessionHour, 'userId', 'outer') \
    .join(f_ErrorPerSessionHour, 'userId', 'outer') \
    .join(f_SettingsPerSessionHour, 'userId', 'outer') \
    .join(f_SaveSettingsPerSessionHour, 'userId', 'outer') \
    .join(f_LogoutPerSessionHour, 'userId', 'outer') \
    .join(f_SubmitDowngradePerSessionHour, 'userId', 'outer') \
    .join(f_RollAdvertPerHour, 'userId', 'outer') \
    .join(f_ThumbsDownPerHour, 'userId', 'outer') \
    .join(f_UpgradePerHour, 'userId', 'outer') \
    .join(f_SubmitUpgradePerHour, 'userId', 'outer') \
    .join(f_SessionsPerDay, 'userId', 'outer') \
    .join(f_AddFriendPerDay, 'userId', 'outer') \
    .join(f_RollAdvertPerDay, 'userId', 'outer') \
    .join(f_ThumbsDownPerDay, 'userId', 'outer') \
    .join(f_ThumbsUpPerDay, 'userId', 'outer') \
    .join(f_TotalSongLength, 'userId', 'outer') \
    .join(f_UniqueSongCount, 'userId', 'outer') \
    .join(f_UniqueSongShare, 'userId', 'outer') \
    .drop('userId') \
    .fillna(0)

### Model fitting by default parameters

##### Standard Scaler

In [272]:
scaler = StandardScaler(inputCol="NumFeatures", outputCol="features")
classifier = RandomForestClassifier(seed=0)
model_rfc_standard = fit_model('Random Forest (Standard Scaler)', data_rfc, classifier, scaler)

[Standard Scaler]

Model fitting started. Please wait...
Train|Test data split done.
Create pipeline done.
Training done.
Metrics - Random Forest
-------------------------------------------------------
 - accuracy: 0.92
 - F1: 0.912888888888889
 - weighted precision: 0.9126086956521738
 - weighted recall: 0.9199999999999999
 - AUC: 0.7386363636363636
-------------------------------------------------------
Model fitting done (1486 s).


##### MinMax Scaler

In [274]:
scaler = MinMaxScaler(inputCol="NumFeatures", outputCol="features")
classifier = RandomForestClassifier(seed=0)
model_rfc_minmax = fit_model('Random Forest (MinMax Scaler)', data_rfc, classifier, scaler)

Model fitting started. Please wait...
Train|Test data split done.
Create pipeline done.
Training done.
Metrics - Random Forest (MinMax Scaler)
-------------------------------------------------------
 - accuracy: 0.92
 - F1: 0.912888888888889
 - weighted precision: 0.9126086956521738
 - weighted recall: 0.9199999999999999
 - AUC: 0.7386363636363636
-------------------------------------------------------
Model fitting done (1536 s).


In [280]:
feature_coeff_rfc = model_rfc_minmax.stages[-1].featureImportances
feature_importances_rfc = pd.DataFrame(list(zip(features, feature_coeff_rf)), columns=['Feature', 'FeatureImportances']) \
    .sort_values('FeatureImportances', ascending=False)
feature_importances_rfc

,Feature,FeatureImportances
10,AvgSessionGap,0.202037
18,ThumbsDownPerHour,0.084404
21,SessionsPerDay,0.082781
17,RollAdvertPerHour,0.063480
24,ThumbsDownPerDay,0.041387
23,RollAdvertPerDay,0.034505
13,SettingsPerSessionHour,0.032447
25,ThumbsUpPerDay,0.031034
11,DowngradePerSessionHour,0.030941
22,AddFriendPerDay,0.030402


### Cross-Validation

In [294]:
print('Cross validation started. Please wait...')

# Create classifier
rfc = RandomForestClassifier(seed=0)

# Create pipeline
scaler = StandardScaler(inputCol="NumFeatures", outputCol="features")
pipeline = create_pipeline(data_rfc, rfc, scaler)

# Define params
paramGrid_rfc = ParamGridBuilder() \
    .addGrid(rfc.maxDepth, [4, 5, 6]) \
    .addGrid(rfc.numTrees, [10, 20, 50]) \
    .build()

# Create cross validator
crossval_rfc = CrossValidator(estimator = pipeline,
                              estimatorParamMaps = paramGrid_rfc,
                              evaluator = MulticlassClassificationEvaluator(),
                              numFolds = 2)

# Fit model
start_time = time.time()
crossval_model_rfc = crossval_rfc.fit(train)
duration = time.time() - start_time

print(f'Cross validation done ({int(duration)} s).')

# Show best params
print('\nCross validation parameters ordered by score, descending:')
scores = crossval_model_rfc.avgMetrics
params = [{p.name: v for p, v in m.items()} for m in crossval_model_rfc.getEstimatorParamMaps()]
params_df = pd.DataFrame(params)
params_df['score'] = scores
params_df.sort_values(by='score', ascending=False)

Cross validation started. Please wait...
Cross validation done (1739 s).

Cross validation parameters ordered by score, descending:


,maxDepth,numTrees,score
7,6,20,0.759634
2,4,50,0.758248
8,6,50,0.756516
3,5,10,0.754577
4,5,20,0.752732
5,5,50,0.748812
6,6,10,0.745256
0,4,10,0.725191
1,4,20,0.722808


In [297]:
scaler = StandardScaler(inputCol="NumFeatures", outputCol="features")
classifier = RandomForestClassifier(seed=0, maxDepth=6, numTrees=20)
model_rfc_cvbest = fit_model('Random Forest (Cross-Validation Best)', data_rfc, classifier, scaler)

Model fitting started. Please wait...
Train|Test data split done.
Create pipeline done.
Training done.
Metrics - Random Forest (Cross-Validation Best)
-------------------------------------------------------
 - accuracy: 0.92
 - F1: 0.912888888888889
 - weighted precision: 0.9126086956521738
 - weighted recall: 0.9199999999999999
 - AUC: 0.7386363636363636
-------------------------------------------------------
Model fitting done (1456 s).


In [302]:
feature_coeff_rfc = model_rfc_cvbest.stages[-1].featureImportances
feature_importances_rfc = pd.DataFrame(list(zip(features, feature_coeff_rf)), columns=['Feature', 'FeatureImportances']) \
    .sort_values('FeatureImportances', ascending=False)
feature_importances_rfc

,Feature,FeatureImportances
10,AvgSessionGap,0.213280
21,SessionsPerDay,0.071358
18,ThumbsDownPerHour,0.061838
17,RollAdvertPerHour,0.057433
8,SessionCount,0.042105
15,LogoutPerSessionHour,0.041166
23,RollAdvertPerDay,0.040237
24,ThumbsDownPerDay,0.037549
7,RollAdvertCount,0.034767
22,AddFriendPerDay,0.034394


# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.